# Performing differential expression analysis with AlphaQuant and investigating the results

Below some code how to perform differential expression testing with AlphaQuant. This means we do a pairwise comparison of biological conditions and some volcano plot visualizations.

## Specifying input files

In order to run differential analyses you need:
* an input file from a proteomics search engine (e.g. DIA-NN, Spectronaut, AlphaPept etc.), or a pre-formatted file
* a sample mapping file that maps each sample to a condition (e.g.  sample 'brain_replicate_1' is mapped to condition 'brain'). In the GUI, there is some functionality to help create such a file
* (optional) a results directory can be defined on where to save the data
* (optional) a list where we specify, which conditions we compare

AlphaQuant is compatible to the default output tables of most common proteomics search engines. Detailed specifications on which tables you need can be found 
in our  [README](https://github.com/MannLabs/alphaquant/blob/main/README.md#section-title).


In [ ]:
INPUT_FILE = "./data/mouse_tissues/example_dataset_mouse_sn.tsv"
SAMPLEMAP_FILE = "./data/mouse_tissues/samplemap_200.tsv"
CONDPAIRS_TO_COMPARE = [('muscle', 'brain'), ('kidney', 'liver')]
RESULTS_DIRECTORY = "./data/mouse_tissues/results_pairwise_analysis"

## Run AlphaQuant in pairwise-comparison mode

Now we can use the 'run_pipeline' command from AlphaQuant. This performs normalization between sample and then differential expression testing. The main output is a table of proteins with fold changes and an fdr value as a result of the differential expression test. Due to AlphaQuant's tree based approach, sensitivity is in general higher as compared to standard approaches like the t-test, i.e. there should be more proteins with significant fdr, especially for lower fold changes.

In [ ]:
import alphaquant.run_pipeline as aq_pipeline

aq_pipeline.run_pipeline(input_file=INPUT_FILE, samplemap_file=SAMPLEMAP_FILE,
                         condpairs_list=CONDPAIRS_TO_COMPARE, results_dir=RESULTS_DIRECTORY, minrep_both=7)

## Inspecting output tables

The main output tables for each pairwise comparison are located in the results folder and always follow the naming scheme `condition1_VS_condition2.results.tsv`
 * muscle_VS_brain.results.tsv
 * kidney_VS_liver.results.tsv

Let's load one of them:
 
 

In [ ]:
import pandas as pd

df_spleen_testis  = pd.read_csv(RESULTS_DIRECTORY + "/muscle_VS_brain.results.tsv", sep='\t')
display(df_spleen_testis)

The fold changes are defined as condition1 - condition2. For example spleen_VS_testis means that a log2fc is calculated by log2(intensity_spleen/intensity_testis)

## Visualizing results - basic plots

### Volcano plot

In [ ]:
import alphaquant.plotting.pairwise as aq_plotting_pairwise

aq_plotting_pairwise.volcano_plot(df_spleen_testis)

### Normalization plot

The normalization plot gives the foldchanges between a given sample and the median over all samples. The peak of the distributions should be centered around 0. If not, there might be a problem with the normalization.

In [ ]:
normalized_df = pd.read_csv(RESULTS_DIRECTORY + "/muscle_VS_brain.normed.tsv", sep='\t')
samplemap_df = pd.read_csv(SAMPLEMAP_FILE, sep='\t')
aq_plotting_pairwise.plot_normalization_overview(normalized_df, samplemap_df)


## Inspect and visualize potential proteoforms

Our approach enables proteoform inference by analyzing co-clustering peptide groups, since modifications and splicing create distinctive quantitative patterns for affected peptides. For example, consider a protein with four copies in two conditions, but varying phosphorylation: one copy phosphorylated in control and three copies phosphorylated in case. Here, the modified peptide shows a ratio of 3:1, while unmodified peptides maintain a 1:1 ratio.We detect these distinctive patterns.

We write out a `.proteoforms.tsv` table for each pair of conditions. Let's load the one for spleen and testis:

In [ ]:
import pandas as pd
proteoform_df = pd.read_csv(RESULTS_DIRECTORY + "/muscle_VS_brain.proteoforms.tsv", sep='\t')
display(proteoform_df)

The proteoform table contains information about groups of co-clustering peptides, where each group potentially represents a distinct proteoform. Here's what each column means:

* *protein*: The protein or gene name
* *proteoform_id*: A unique identifier combining the protein name and a numerical suffix (e.g., "Mapt_0", "Mapt_1"). The suffix indicates different proteoforms of the same protein
* *cluster*: A numerical identifier for each distinct peptide cluster
* *is_reference*: Boolean flag (TRUE/FALSE) indicating whether this is the reference proteoform. The proteoform with the highest quality_score is designated as the reference
* *peptides*: List of peptide sequences belonging to this proteoform
* *quality_score*: A measure of confidence in the proteoform identification (range: 0-1, higher is better)
* *log2fc*: The calculated log2 fold change between conditions for this proteoform
* *fraction_of_peptides*: The proportion of the protein's total peptides that belong to this proteoform
* *fcdiff*: The difference in fold change compared to the reference proteoform

Note: Most proteins will have only one proteoform (the reference proteoform).

Now, let's filter the table to interesting candidates that have a low proteoform_fdr and at least 2 proteoform_ids 

In [ ]:
import alphaquant.utils.proteoform_utils as aq_utils_proteoforms

proteoform_df_filtered = aq_utils_proteoforms.filter_proteoform_df(proteoform_df=proteoform_df, min_num_peptides=1)
display(proteoform_df_filtered)

This leaves only a two proteins of interest. Let's visualize them in order to check them out further. We will first visualize the fold changes of all the peptides of the proteins of interest. Peptides of the same proteoform_id are grouped together. This way, we can understand how the model grouped the peptides into proteoform_ids.

In [ ]:
import alphaquant.plotting.fcviz as aq_plot_fc

proteins_of_interest = proteoform_df_filtered['protein'].unique()
print(proteins_of_interest)
fc_visualizer = aq_plot_fc.FoldChangeVisualizer(condition1="muscle", condition2="brain", results_directory=RESULTS_DIRECTORY, samplemap_file=SAMPLEMAP_FILE)

fc_visualizer.plot_list_of_proteins(proteins_of_interest)

Of those two, Mapt looks the most interesting, are there are multiple very consistent peptides. Let's use the AlphaMap package (Voytik et al.), which is integrated in AlphaQuant, to visualize the proteoform.

In [ ]:
import alphaquant.plotting.alphamapviz as aq_plot_alphamap

amap_visualizer = aq_plot_alphamap.AlphaMapVisualizer(condition1="muscle", condition2="brain", results_directory=RESULTS_DIRECTORY, samplemap_file=SAMPLEMAP_FILE,protein_identifier='gene_symbol', organism="Mouse") #because we need to map the protein to it's respective sequence, we need to specify that the protein identidier is uniprot and the organism is Mouse

In [ ]:
fc_plot, alphamap_plot = amap_visualizer.visualize_protein("Mapt")

fc_plot.show()
display(alphamap_plot)

Indeed we see that four of the Mapt_1 peptides seem to occupy one domain of the  protein sequence. Interestingly, the two peptides in the beginning (SEQ_AEEAGIGDTPN.. and SEQ_VEIFSQS) do not map to the sequence displayed in alphamap. This is a strong indication of alternative splicing happening on different isoforms of Mapt.